 Import Necessary Libraries

In [1]:
# Import necessary libraries
import zipfile
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

KeyboardInterrupt: 

In [ ]:
import os
from PIL import Image, ImageFile
from torchvision import datasets, transforms, models
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import seaborn as sns
import pandas as pd

Mount Google Drive and Extract ZIP file

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the ZIP file in your Google Drive
zip_file_path = '/content/drive/MyDrive/ADATELEMZÉS_HF/top_100_species_images.zip'

Load Images and Extract Labels

In [ ]:
#### Load the dataset from CSV file
df = pd.read_csv('/content/drive/MyDrive/ADATELEMZÉS_HF/FungiCLEF2023_train_metadata_PRODUCTION.csv')

#### Find the top 100 species by number of examples
top_100_species = df['species'].value_counts().head(100).index.tolist()

#### Filter the dataframe to only include rows from the top 100 species
df_top_100 = df[df['species'].isin(top_100_species)]

#### Create a new dataframe with only the 'image_path' and 'class_id' columns
top_100_pairs = df_top_100[['image_path', 'species']]

#### Print the shape of the final dataframe
print(f"Final dataframe shape: {top_100_pairs.shape}")

unique_species_ids = top_100_pairs['species'].nunique()
print(f"Number of unique species: {unique_species_ids}")
print(top_100_species)

# Create a dictionary that maps each species to a unique integer label
species_labels = {species: index for index, species in enumerate(top_100_species)}

print(species_labels)

In [ ]:
from torchvision import transforms

# Define transformations to be applied to each image (resizing, converting to tensor, normalizing)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 288x288 pixels
    transforms.ToTensor(),  # Convert image to a tensor (used by PyTorch)
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import zipfile

# Define a custom dataset class for loading our images and labels
class CustomDataset(Dataset):
    def __init__(self, zip_file_path, top_100_df, transform=None):
        self.zip_file_path = zip_file_path
        self.top_100_df = top_100_df
        self.transform = transform
        self.image_paths = []  # List to store image paths
        self.labels = []       # List to store image labels
        image_count = 0
        label_count = 0
        iter = 0

        print("Zip folder path:", zip_file_path)  # Print the folder path

        # Open the ZIP file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
            # Get the name of the folder
            folder_name = os.path.splitext(os.path.basename(zip_file_path))[0]
            print("Folder name:", folder_name)  # Print the subfolder name

            # Loop through each file in the ZIP file
            for file_info in zip_file.infolist():
                file_name = file_info.filename
                # print("File name:", file_name)  # Print the file name

                if file_name.startswith(folder_name + '/') and file_name.endswith('.JPG') and not file_name.startswith('__MACOSX/'):
                    #print("Matched file:", file_name)  # Print the matched file name

                    if(iter % 10 == 0):
                        # Get the image path after the '/' symbol
                        image_path_after_slash = os.path.basename(file_name)
                        #print("Needed name: ", image_path_after_slash)

                        # Extract the label from the file name
                        label = species_labels.get(self.top_100_df.loc[self.top_100_df['image_path'] == image_path_after_slash, 'species'].values[0])
                        # print("Extracted label:", label)  # Print the extracted label

                        self.image_paths.append(file_name)  # Store the image path within the ZIP file
                        image_count += 1
                        self.labels.append(label)  # Store the corresponding label
                        label_count += 1

                    iter = iter + 1

                    if(label_count % 100 == 0):
                      print(label_count)

        # print("Image paths:", self.image_paths)
        # print("Labels:", self.labels)
        print(len(set(self.labels)))
        print("ZIP file path:", zip_file_path)  # Print the ZIP file path
        print("Subfolder name:", folder_name)  # Print the subfolder name
        print("Image count:", image_count)
        print("Label count:", label_count)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Open the ZIP file and read the image
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_file:
            img_path = self.image_paths[idx]
            image = Image.open(zip_file.open(img_path)).convert("RGB")
            label = self.labels[idx]

            if self.transform:
                image = self.transform(image)

        return image, label

In [ ]:
# Create dataset and dataloader objects for the training dataset
dataset = CustomDataset(zip_file_path=zip_file_path, top_100_df=top_100_pairs, transform=transform)

# Create DataLoaders to efficiently load data in batches
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
# Define the model using pre-trained ResNet-50
class MushroomClassifier(nn.Module):
    def __init__(self, num_classes=100):
        super(MushroomClassifier, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Instantiate the model and move it to GPU if available
num_classes = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MushroomClassifier(num_classes=num_classes).to(device)

In [ ]:
# Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# KFold keresztvalidációs trainelése a modellnek

In [ ]:

from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, SubsetRandomSampler

# Function to reset model weights
def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

# Define the train_kfold function
def train_kfold(model, dataset, batch_size, num_folds, num_epochs, optimizer, criterion, device):
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"Fold {fold + 1}/{num_folds}")

        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)

        train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

        model.apply(reset_weights)

        for epoch in range(num_epochs):
            print("Itt vagyok epochban:", epoch + 1)
            model.train()
            train_loss, train_acc = 0.0, 0.0

            for batch in train_loader:
                print("Itt vagyok batchben:", batch + 1)
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                train_acc += torch.sum(preds == targets).item()

            train_loss /= len(train_loader)
            train_acc /= len(train_idx)

            model.eval()
            val_loss, val_acc = 0.0, 0.0

            with torch.no_grad():
                for batch in val_loader:
                    inputs, targets = batch
                    inputs, targets = inputs.to(device), targets.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_loss += loss.item()

                    _, preds = torch.max(outputs, 1)
                    val_acc += torch.sum(preds == targets).item()

            val_loss /= len(val_loader)
            val_acc /= len(val_idx)

            print(f"Epoch {epoch + 1}/{num_epochs} | "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)

    return train_losses, val_losses, train_accuracies, val_accuracies


In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies = train_kfold(
    model=model,
    dataset=dataset,
    batch_size= 16,
    num_folds= 5,
    num_epochs= 10,
    optimizer=optimizer,
    criterion=criterion,
    device=device
)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

y_pred = model.predict(X_test)

y_pred_classes = y_pred.argmax(axis=1)

macro_f1 = f1_score(y_test, y_pred_classes, average='macro')
print(f"Macro F1 score: {macro_f1:.4f}")

accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

print("Classification Report:\n", classification_report(y_test, y_pred_classes))
